In [ ]:
# ============================================================================
# CONFIGURATION: Choose which simulator configuration to use
# Change 'simulator' to 'simulator_paper' to switch configurations
# ============================================================================
SIMULATOR_MODULE = 'simulator'  # Options: 'simulator' or 'simulator_paper'
# ============================================================================


# EDA for Baseline Performance
### Setup and import baseline policies data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import importlib

# Add project root to path to import your custom policies
sys.path.append(os.path.abspath('..'))
from scripts.baseline_policies import (
    policy_always_d2d, policy_always_cellular, 
    policy_random, policy_sinr_threshold, policy_ground_truth
)

# Dynamically import the config from the selected simulator module
config_module = importlib.import_module(f'{SIMULATOR_MODULE}.config')
SimulationConfig = config_module.SimulationConfig
if hasattr(config_module, 'PaperConfig'):
    PaperConfig = config_module.PaperConfig

print(f"✓ Loaded configuration from '{SIMULATOR_MODULE}'")

%matplotlib inline
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

### Load baseline policies data

In [ ]:
# Load the summary results
summary_path = os.path.join('..', 'data', 'baseline_results.csv')

if os.path.exists(summary_path):
    df_results = pd.read_csv(summary_path)
    print("Loaded Baseline Results:")
    display(df_results)
else:
    print("Error: Could not find data/baseline_results.csv. Run 'evaluate_baselines.py' first.")

### Visualize baseline performance metrics

In [ ]:
if os.path.exists(summary_path):
    # Create side-by-side subplots
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Plot 1: Average Throughput (The "Performance" Metric)
    sns.barplot(x='Policy', y='Avg Throughput (Mbps)', data=df_results, ax=axes[0], palette='viridis')
    axes[0].set_title('Average Throughput Comparison', fontsize=14)
    axes[0].set_ylabel('Throughput (Mbps)')
    axes[0].tick_params(axis='x', rotation=45) # Rotate labels to fit
    
    # Plot 2: Switching Rate (The "Cost" Metric)
    sns.barplot(x='Policy', y='Switching Rate (per 100s)', data=df_results, ax=axes[1], palette='magma')
    axes[1].set_title('Switching Rate Comparison (Stability)', fontsize=14)
    axes[1].set_ylabel('Switches per 100s')
    axes[1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()

### Timeline analyis for Episode 0 only

In [ ]:
# 1. Load the raw simulation data
raw_path = os.path.join('..', SimulationConfig.OUTPUT_FILE)
df_raw = pd.read_csv(raw_path)

# 2. Select ONLY Episode 0 (The first run)
episode_0 = df_raw[df_raw['episode_id'] == 0].copy().reset_index(drop=True)

# 3. Re-run the policies on this single episode
ep0_oracle = policy_ground_truth(episode_0)
ep0_threshold = policy_sinr_threshold(episode_0, threshold_db=0)
ep0_random = policy_random(episode_0)

# 4. Convert "D2D"/"Cellular" text to numbers (1/0) for plotting
# D2D = 1, Cellular = 0
trace_oracle = ep0_oracle.apply(lambda x: 1 if x == 'D2D' else 0)
trace_threshold = ep0_threshold.apply(lambda x: 1 if x == 'D2D' else 0)
trace_random = ep0_random.apply(lambda x: 1 if x == 'D2D' else 0)

# 5. Plot the Timeline
plt.figure(figsize=(15, 4))

# Plot Oracle (Ground Truth)
plt.step(episode_0['timestamp'], trace_oracle, where='post', label='Oracle (Optimal)', linewidth=2, color='green')

# Plot Threshold (Heuristic) -> Shifted slightly up so we can see it
plt.step(episode_0['timestamp'], trace_threshold + 0.05, where='post', label='SINR Threshold', linewidth=2, linestyle='--', color='blue')

# Plot SINR on a secondary axis to explain WHY it switched
plt.ylabel('Mode (0=Cell, 1=D2D)')
plt.yticks([0, 1], ['Cellular', 'D2D'])
plt.title('Policy Decisions over Time (Episode 0)', fontsize=14)
plt.legend(loc='center left')   

# Add SINR context
ax2 = plt.gca().twinx()
ax2.plot(episode_0['timestamp'], episode_0['sinr_d2d_db'], color='gray', alpha=0.3, label='D2D SINR (dB)')
ax2.axhline(0, color='red', linestyle=':', alpha=0.5, label='0dB Threshold')
ax2.set_ylabel('D2D SINR (dB)', color='gray')

plt.xlim(0, 100)
plt.show()

### Comment 1
The high amount of switching for the ground truth policy shows why it has a high switching rate (~30%). It can also be observed that the for SINR threshold policy, it only switched mode when the D2D SINR is above 0 dB. This indicates wasted potential where there are many instances where D2D mode gives a higher throughput, but the SINR threshold policy missed them many times. This problem will be solved using the deep learning models. 